# 06: NGrams - Basic Natural Language Processing

In *Natural Language Processing*, one goal is to determine the sentiment or meaning of text. One technique that helps do this is to locate the most frequently-occurring, N-word phrases, or *NGrams*. Longer NGrams can convey more meaning, but they occur less frequently so few of them stand out as important. Shorter NGrams have better statistics, but each one conveys less meaning. In most cases, `N = 3-5` appears to provide the best balance.

This exercise finds all NGrams matching a user-specified pattern. The default is the 4-word phrases the form `% love % %`, where the `%` are wild cards and the spaces ` ` will be treated as matches for any whitespace (but not crossing line boundaries). In other words, all 4-grams are found with `love` as the second word. The `%` are conveniences; the user can also specify an NGram Phrase that is a regular expression or a mixture, e.g., `% (hat|lov)ed? % %` finds all the phrases with `love`, `loved`, `hate`, or `hated` as the second word. 

> **Note:** All text is converted to lower case (but see the comment where you could change that). The ngrams matching string is not converted, so use lower case words!

See the corresponding Spark job [NGrams6.scala](https://github.com/deanwampler/spark-scala-tutorial/blob/master/src/main/scala/sparktutorial/NGrams6.scala).

In [1]:
val in = "../data/shakespeare/all-shakespeare.txt"  // The plays of Shakespeare!
val count = 100               // Upper limit on how many ngrams to return

in = ../data/shakespeare/all-shakespeare.txt
count = 100


100

In [2]:
val ngramsStr = "% love % %"    // Edit to define what ngrams you want to see.

// Convert the string into a valid regex:
val ngramsRE = ngramsStr.replaceAll("%", """\\w+""").replaceAll("\\s+", """\\s+""").r

ngramsStr = % love % %
ngramsRE = \w+\s+love\s+\w+\s+\w+


\w+\s+love\s+\w+\s+\w+

Recall this helper function from 03_WordCount. 

> **Note:** If you don't want the text converted to lower case, remove the call to `toLowerCase`.

In [3]:
def toText(str: String): String = {
  val ary = str.toLowerCase.split("\\s*\\|\\s*")
  if (ary.length > 0) ary.last else ""
}

toText: (str: String)String


Define an object for sorting. We'll want to sort by count, _descending_, but when two counts are equal, we'll _secondary sort_ by the ngram itself. The latter step is mostly useful for repeatable unit tests! 

In [4]:
object CountOrdering extends Ordering[(String,Int)] {
  def compare(a:(String,Int), b:(String,Int)) = {
    val cntdiff = b._2 compare a._2    // b compare a for these counts means DESCENDING order
    if (cntdiff != 0) cntdiff else (a._1 compare b._1)
  }
}

defined object CountOrdering


Now extract the NGrams:

1. Read the text file(s).
2. Parse each line into words.
3. For each line, find all matches for the ngrams regex.
4. Do _Word Count_, but with each NGram as a key, not each word.
5. Use `takeOrdered`, an efficient function that takes the top `count` records, according to the ordering defined by `CountOrdering`. Because it does descending count ordering, we'll return the `count` most frequently occuring ngrams!

Note that 3. implies that we don't match across line boundaries. (See the exercises.)

In [5]:
val ngrams = sc.textFile(in)
  .flatMap { line =>
    val text = toText(line)
    ngramsRE.findAllMatchIn(text).map(_.toString)   // Find matches only per line. Return matching strings
  }
  .map(ngram => (ngram, 1))                         // Word Count, but using the ngrams
  .reduceByKey(_ + _)
  .takeOrdered(count)(CountOrdering)                // Take the top "count" most frequently occurring ngrams

ngrams = Array((i love thee not,7), (the love i bear,4), (you love me not,4), (i love and honour,3), (i love him not,3), (i love him well,3), (i love thee better,3), (i love thee in,3), (i love you not,3), (in love with him,3), (my love to thee,3), (for love of her,2), (for love of you,2), (he love her not,2), (i love my country,2), (i love not to,2), (i love the king,2), (i love thee more,2), (i love thee well,2), (i love thy daughter,2), (i love to hear,2), (i love you more,2), (i love you the,2), (if love be blind,2), (if love make me,2), (in love or that,2), (in love with a,2), (in love with beatrice,2), (in love with her,2), (in love with me,2), (in love with my,2), (in love with thee,2), (my love and duty,2), (my love is as,2), (my love swears that,2), (my lo...


[(i love thee not,7), (the love i bear,4), (you love me not,4), (i love and honour,3), (i love him not,3), (i love him well,3), (i love thee better,3), (i love thee in,3), (i love you not,3), (in love with him,3), (my love to thee,3), (for love of her,2), (for love of you,2), (he love her not,2), (i love my country,2), (i love not to,2), (i love the king,2), (i love thee more,2), (i love thee well,2), (i love thy daughter,2), (i love to hear,2), (i love you more,2), (i love you the,2), (if love be blind,2), (if love make me,2), (in love or that,2), (in love with a,2), (in love with beatrice,2), (in love with her,2), (in love with me,2), (in love with my,2), (in love with thee,2), (my love and duty,2), (my love is as,2), (my love swears that,2), (my love to her,2), (my love to you,2), (of love i mean,2), (of love in her,2), (the love i bore,2), (the love of god,2), (the love of laughter,2), (thy love to me,2), (you love my son,2), (you love the gentleman,2), (you love the maid,2), (your

Note that we could have used `.sortByKey(descending = false).take(n)`, but it's much less efficient to sort the whole data set, then throw away most of it!

Note that `takeOrdered` returns a Scala `Array[(String,Int)]`, not an `RDD`.

Let's format the output a little better:

In [6]:
val outputLines = s"Found ${ngrams.size} ngrams:" +:
  ngrams.map {
    case (ngram, count) => "%30s\t%d".format(ngram, count)
  }

outputLines = Array(Found 100 ngrams:, "               i love thee not	7", "               the love i bear	4", "               you love me not	4", "             i love and honour	3", "                i love him not	3", "               i love him well	3", "            i love thee better	3", "                i love thee in	3", "                i love you not	3", "              in love with him	3", "               my love to thee	3", "               for love of her	2", "               for love of you	2", "               he love her not	2", "             i love my country	2", "                 i love not to	2", "               i love the king	2", "              i love thee more	2", "              i love thee well	2", "           i love thy daughter	2", "                i love...


[Found 100 ngrams:,                i love thee not	7,                the love i bear	4,                you love me not	4,              i love and honour	3,                 i love him not	3,                i love him well	3,             i love thee better	3,                 i love thee in	3,                 i love you not	3,               in love with him	3,                my love to thee	3,                for love of her	2,                for love of you	2,                he love her not	2,              i love my country	2,                  i love not to	2,                i love the king	2,               i love thee more	2,               i love thee well	2,            i love thy daughter	2,                 i love to hear	2,                i love you more	2,                 i love you the	2,               if love be blind	2,                if love make me	2,                in love or that	2,                 in love with a	2,          in love with beatrice	2,               in love with h

Let's print the first 20 a bit more legibly.

In [7]:
outputLines.take(20).foreach(println)

Found 100 ngrams:
               i love thee not	7
               the love i bear	4
               you love me not	4
             i love and honour	3
                i love him not	3
               i love him well	3
            i love thee better	3
                i love thee in	3
                i love you not	3
              in love with him	3
               my love to thee	3
               for love of her	2
               for love of you	2
               he love her not	2
             i love my country	2
                 i love not to	2
               i love the king	2
              i love thee more	2
              i love thee well	2


Clearly, many people were _not_ in love with each other in Shakespeare!

Save the output, if you want. We've seen most of what's interesting and if you save the output, then rerun, you'll have to either delete the old output or specify a different output directory:

In [8]:
val out = "output/ngrams"
val output = sc.makeRDD(outputLines)  // convert back to an RDD
output.saveAsTextFile(out)

out = output/ngrams
output = ParallelCollectionRDD[6] at makeRDD at <console>:42


ParallelCollectionRDD[6] at makeRDD at <console>:42

## Recap

NGrams are not only useful, they can be fun to play with...

## Exercises

### Exercise 1: Try different input texts and NGram specifications

For example, try the `% (hat|lov)ed? % %` specification. Try the Bible texts.

### Exercise 2: Match NGrams across lines

This is harder, as it's difficult to create an RDD of lines, then match across lines. However, try using the same code we used for the `Crawl` step in notebook 5 for the _Inverted Index_. Recall that we used a method of reading a whole file as a single record. That will work fine as long as no file is too big.